In [138]:
import pandas as pd
import numpy as np
# from pandas_profiling import ProfileReport
from ydata_profiling import ProfileReport

In [141]:
df = pd.read_csv("/home/gui/Área de Trabalho/DS_challenge/DS_challenge/SourceFiles/ofertas_relampago.csv").head()
df.head()

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID
0,2021-06-22,2021-06-22 16:00:00+00:00,2021-06-22 23:02:43+00:00,lightning_deal,4,-2,4.72,6.0,A,none,PETS FOOD,CPG,MLM-BIRD_FOODS
1,2021-06-22,2021-06-22 13:00:00+00:00,2021-06-22 19:00:02+00:00,lightning_deal,5,5,NaN,NaN,NaN,free_shipping,PET PRODUCTS,OTHERS,MLM-ANIMAL_AND_PET_PRODUCTS
2,2021-06-22,2021-06-22 07:00:00+00:00,2021-06-22 13:00:01+00:00,lightning_deal,15,12,10.73,3.0,NaN,none,COMPUTERS,CE,MLM-SPEAKERS
3,2021-06-22,2021-06-22 19:00:00+00:00,2021-06-23 01:36:12+00:00,lightning_deal,15,13,7.03,2.0,NaN,none,COMPUTERS,CE,MLM-HEADPHONES
4,2021-06-22,2021-06-22 13:00:00+00:00,2021-06-22 15:48:12+00:00,lightning_deal,15,0,39.65,15.0,NaN,none,COMPUTERS,CE,MLM-HEADPHONES


In [3]:
profile = ProfileReport(df, title="My Data Profile")
# profile.to_widgets()
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

# Primeiras Ideias 
 - _OFFER_TYPE_ parece ser um identificador para as "ofertas relâpago" e apresenta um valor constante neste dataset, portanto não usarei nas análises
 - _ORIGIN_ também parece ser algum tipo de identificador, menos de 1 quarto dos dados aprensenta um valor e sempre o mesmo, não sei dizer o que significa, também não usarei nas análises
 - _OFFER_START_DATE_ possui um range de 2 meses apenas e nenhum valor faltante, podemos criar outra variável ao identificar os dias da semana, e analisar o TPV em finais de semana 
 - _OFFER_START_DTTM_ e _OFFER_FINISH_DTTM_ trazem uma janela em que a oferta ficou disponível, podemos analisar o impacto do tamanho dessa janela no TPV, seria interessante ver o volume de vendas distribuído ao decorrer da janela (será que as vendas aumentam próximo do fim?). Também podemos analizar o volume de vendas em diferentes períodos, ex. fim de tarde, e fora de horário comercial. 
 - _REMAINING_STOCK_AFTER_END_ esta variável possui valores negativos, isso sugere que houve mais vendas do que havia de produtos em estoque? qual seria o impacto disso no negócio? podemos analizar se há um padrão nos produtos/janelas que isso ocorre.
 - _SOLD_AMOUNT_ e _SOLD_QUANTITY_ serão uteis pra calcular o TPV de ofertas, e podemos utilizar pra quantificar o sucesso de produtos/janelas de ofertas.
 - _SHIPPING_PAYMENT_TYPE_ parece interessante para correlacionar com produtos, mas o domínio é muito pequeno sendo _free_shipping_ o único valor presnte, não sei dizer se a falta de valor corresponde necessáriamente a um valor diferente de _free_shipping_.
 - _DOM_DOMAIN_AGG1_ e _VERTICAL_ parecem ser  categorias hierárquicas dos produtos, podemos identificar as verticais de produtos em que as ofertas relampago conseguem uma maior aderência.
- DOMAIN_ID parece identificar um tipo de produto específico, possui um domínio bem grande.


In [143]:
df['OFFER_DATE'] = pd.to_datetime(df['OFFER_START_DATE'])
df['OFFER_WEEK_DAY'] = df.OFFER_DATE.dt.weekday # Monday is 0 and Sunday is 6
df['OFFER_WEEKEND'] = df.OFFER_WEEK_DAY.apply(lambda x: x in (5,6))

df['OFFER_LENGTH_IN_HOURS'] = (pd.to_datetime(df['OFFER_FINISH_DTTM']) - pd.to_datetime(df['OFFER_START_DTTM'])).dt.total_seconds() / 3600
# df['OFFER_HOURS'] = df.apply(lambda x: [pd.to_datetime(x.OFFER_START_DTTM).hour + h for h in range(round(x.OFFER_LENGTH_IN_HOURS)+1)], axis=1)

df['OFFER_LENGTH_IN_HOURS']
# df.OFFER_LENGTH_IN_HOURS.describe()

0    7.045278
1    6.000556
2    6.000278
3    6.603333
4    2.803333
Name: OFFER_LENGTH_IN_HOURS, dtype: float64

In [151]:
labels = ['NIGHT', 'MORNING', 'AFTERNOON', 'EVENING']
bins = [0, 6, 12, 18, 24]

def time_of_day_percentage(x):
    start_time = pd.to_datetime(x.OFFER_START_DTTM)
    duration = x.OFFER_LENGTH_IN_HOURS
    durationR = round(duration)
    hours = [start_time.hour + h for h in range(durationR+1)]
    remain = duration - durationR

    # print(start_time, end_time)
    # print(duration)
    # print(remain)
    part_of_day = [
        pd.cut([current_hour], bins=bins, labels=labels, right=False, include_lowest=True)[0]
        for current_hour in hours
    ]

    percentages = {
        label: round(sum([1 if i<(len(part_of_day)-1) else remain for i,p in enumerate(part_of_day) if p ==label])/duration, 3)
        for label in labels
    }
    # print(percentages)
    return percentages

time_of_day_percentage(df.iloc[0])

{'NIGHT': 0.0, 'MORNING': 0.0, 'AFTERNOON': 0.284, 'EVENING': 0.716}

In [152]:
for l in labels:
    df[f'OFFER_PERCENTAGE_AT_{l}'] = df.apply(lambda x: time_of_day_percentage(x)[l], axis=1)

df['OFFER_PERCENTAGE_AT_AFTERNOON']

0    0.284
1    0.833
2    0.167
3    0.000
4    1.000
Name: OFFER_PERCENTAGE_AT_AFTERNOON, dtype: float64

In [153]:
df.iloc[0]

OFFER_START_DATE                                2021-06-22
OFFER_START_DTTM                 2021-06-22 16:00:00+00:00
OFFER_FINISH_DTTM                2021-06-22 23:02:43+00:00
OFFER_TYPE                                  lightning_deal
INVOLVED_STOCK                                           4
REMAINING_STOCK_AFTER_END                               -2
SOLD_AMOUNT                                           4.72
SOLD_QUANTITY                                          6.0
ORIGIN                                                   A
SHIPPING_PAYMENT_TYPE                                 none
DOM_DOMAIN_AGG1                                  PETS FOOD
VERTICAL                                               CPG
DOMAIN_ID                                   MLM-BIRD_FOODS
OFFER_DATE                             2021-06-22 00:00:00
OFFER_WEEK_DAY                                           1
OFFER_WEEKEND                                        False
OFFER_LENGTH_IN_HOURS                             7.0452